<a href="https://colab.research.google.com/github/Nirmal-Narasimha/FinalYRproject/blob/main/Webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from fbprophet import Prophet
from datetime import datetime

In [106]:
series = pd.read_csv('Final Dataset.csv')
series['Datetime'] = pd.to_datetime(series['Datetime'])
series=series.set_index('Datetime')
series_daily=series.resample('D').sum()
holt = ExponentialSmoothing(series_daily['BESCOM'],seasonal='add',seasonal_periods=365).fit()

In [107]:
df = pd.read_csv('Final Dataset.csv')
df.columns=["DT","BESCOM"]
df['DT'] = pd.to_datetime(df['DT'])
df.index = df['DT']
df = df.drop(columns = "DT")
df=df.resample('D').sum()
df.reset_index() \
    .rename(columns={'DT':'ds',
                     'BESCOM':'y'}).head()
# Setup and train model
prophet = Prophet()
prophet.fit(df.reset_index() \
              .rename(columns={'DT':'ds',
                               'BESCOM':'y'}))


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [108]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
!pip install pyngrok==4.1.1
!ngrok authtoken 'XXx'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [110]:
from flask import *
from flask_ngrok import run_with_ngrok

In [111]:
app = Flask(__name__)

In [112]:
@app.route('/')
def home():
    return render_template("index.html")


@app.route('/prediction',methods=['POST'])
def prediction():
    date_format = "%Y-%m-%d"
    ourdate = datetime.strptime(
    request.form['date'],date_format)
    datasetEndDate = datetime.strptime('2018-12-31',date_format)
    delta = ourdate-datasetEndDate
    period = delta.days
    predictions = holt.forecast(period)
    op = predictions[period-1]

    mydate = ourdate.strftime('%Y-%m-%d')
    predict_df = list()
    predict_df.append(mydate)
    predict_df = pd.DataFrame(predict_df)
    predict_df.columns = ['ds']
    predict_df['ds'] = pd.to_datetime(mydate,yearfirst=True)

    forecast = prophet.predict(predict_df)
    forecast = forecast['yhat'][0]
    return render_template('index.html', prediction_text='The power needed on {} is :{}MegaWatts using holtWintersModel \n and {}MegaWatts using prophetModel'.format(ourdate,op,forecast))

   


In [113]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d328-35-221-179-138.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Jun/2022 08:59:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 08:59:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 08:59:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 08:59:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 08:59:48] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 08:59:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2022 08:59:55] "POST /prediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 08:59:55] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 09:02:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 09:02:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 09:02:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 09:02:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2022 09:03:17] "POST /prediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jun/2022 09:03:17] "POST /p